## SLAM-Particle Filter for Qubits under Dephasing

The purpose of this workbook is to apply *qslam* to infer a uniform dephasing noise field from single-qubit Ramsey measurements performed on a grid of qubits.

First, we import the relevant packages

In [ ]:
import sys, os
sys.path.append('../')
from qslam.preamble import *

#### Physical Device 
We assume that a physical device consists of a fixed grid of qubits. The location of a qubit on the grid is known and described in terms of grid-coordinates. For example, we specify a 5x5. grid of 20 qubits below: 

In [ ]:
x_grid=5
y_grid=5
devicegrid = [x_grid, y_grid] # only square grids for now

#### True Dephasing

We now create a  dephasing noise field over this grid. In simulation, at each grid point, we define the derivative of the phase noise field that needs to be inferred using single qubit measurements:

In [ ]:
somemap_ = np.ones((x_grid, y_grid))*np.pi*0.8 # uniform map, should yield Ramsey measurements 0 more frequently than 1
truth = TrueMap(m_vals=somemap_)

#### Control Positions and Scan Ranges

Next, we want to set up a measurement protocol. In code, this means we specify the sequence in which qubits are made available for a sensing measurement. We call these 'controls' as one can imagine that the availbility of qubits on the grid is analogous to telling a 'robot' where to go next in a SLAM framework. 

In [ ]:
controls = [(2, 2, 2.)]*3 
print("Control sequence will take measurements at the following locations:")
print [(idx[0], idx[1]) for idx in controls]

Why is there a third parameter in each control? The third parameter refers to how measurement information at a location (x,y) is "blurred" over the neighbours of (x,y). This is called a 'scan range' - a correlation length that spreads information under the assumption that dephasing noise fields are continuously varying in space. For the controls above, the scan ranges are:

In [ ]:
print("Control sequence will blurr information to nearest neighbours within a correlation length:")
print [(idx[2]) for idx in controls]

As more measurements are taken, a histogram of qubit measurements at each location is the sample-estimate of the true Born probability of observing a '1' on the qubit, and hence can be used to infer the dephasing noise field. However, we want to minimise the amount of measurements taken, and 'blurring' a data point over its neighbours allows us to use 'quasi-measurement' information to infer true dephasing noise. 

#### Initiating SLAM using a Particle Filter

The true state being estimated is the qubit state under a dephasing noise field, namely, we wish to estimate and track the derivative of the phase noise field and compare it to the true map specified for the device.

qslam has been implemented as a particle filter to accomodate non-linear system descriptions and to accomodate the quasi-measurement (blurring) feature. A particle filter requires the user to specify the total number of particles used to approximate the probability distribution (posterior) describing the true state of the system. Changing the number of particles in a particle filter improves the variance on the true state estimates - for a test run with uniform dephasing, it's sufficient to get true state estimates with only 1 particle.

In [ ]:
particle_num=1

The SLAM framework is initiated for a given device as follows:

In [ ]:
slamize = ParticleFilter(num_p=particle_num, localgridcoords_=devicegrid)

The code above sets up a particle filter SLAM framework. 

In this framework:

1. Phyiscal Device :: *devicegrid* :: defined by grid coordinates; a qubit sits on each grid node. Currently only square grids considered. 
2. Sensing Qubit :: *globalbot* :: defined as a **robot**, with a **map** and **scanner**. Robot positions are given as grid coordinates; availability of a sensing qubit is modelled as the movement of a robot from one location to another, specified by *controls* (perfectly). Scanner makes a single physical measurement at a location; then uses the physical measurement to generate a number of quasi-measurements over its nearest neighbours. Physical and quasi measurements are passed on the particle filter. (We assumes that only one sensing qubit is available on the grid at any given time.) The map of the Sensing Qubit is updated using only physical measurements (no quasi-msmts are used).  
3. Particle Filter :: *single particle* :: defined as a **robot**, with a **map**, and a one-step-ahead **predictor**. Robot (particle) position is inherited from the Sensing Qubit. Particle map and predictor are used to make one-step ahead predictions at the current location and its nearest neighbours. This predicted region is compared with physical msmt and quasi measurement data from the Sensing Qubit. The residuals over the entire region are used to calculate a weight (likelihood) for the robot (particle). 
4. Particle Filter :: *posterior map* :: returns posterior map. State information is combined across particles - such that a highly likely particles contribute more information to the overall posterior. 

The above framework is completely general. Note that initiation only requires the user to specify the number of particles.

#### Running qslam

We run qslam using the *qslam_run* method, for a simulated dephasing noise map and a given control path:

In [ ]:
%%timeit
slamize.qslam_run(truth, controls) # predictions function can't handle rectangular grids

#### Results

The key check for any simulation is that the map of Sensing Qubit (*globalbot*, based on physical measurements) and posterior map of Particle Filter are identical in the limit of infinite measurements and zero quasi-measurement scan range. 

For finite, minimal measurements, quasi-measurement data allows particle filter to infer map faster.

Below, we plot the residual squared error maps (not RMS) for (a) globalbot and (b) posterior map from the particle filter, with respect to the true maps. Both should converge to zero error for infinite measurements, but the rate of convergence should be faster for (b). 

In [ ]:
# Global Bot
sample_born_est = np.asarray([slamize.particles[0].get_phase_method(val) for val in slamize.global_bot.r_questbk.flatten()]).reshape(x_grid,y_grid)
resid_globalbot = (truth - sample_born_est)**2

# Particle Map
for idx in range(particle_num):
    posterior += (1./particle_num)*slamize.particles[idx].m_vals
resid_globalbot = (truth - posterior)**2


'''
TEST SCRIPT

Type: Module testing
Original Script: slampf.py
Class: slampf.py
Methods: ALL
Details: Test slam output for single particle, single control, uniform map 
Outcome: SUCCESSFUL

1. Particle maps aren't updating  - FIXED
2. The guestbook for each particle is updating twice on each real msmt - FIXED
3. Add extended print statements to probe single particle, single control SLAM - DONE

Theory Qns: 
1. Are we doing better than randomly measuring qubits on the grid?
2. Is the likelihood function correct?
3. What is the threshold to stop the algorithm?
4. What happens if we average over the same region again and again? Do we lose info by averaging using the scanner?
5. What if we visit a qubit and it's four neighbours? Does the averaging process give us more or less info? 
6. For (5), is there a combination of physical and quasi msmts that are optimal? 
7. How does increasing the number of particles reduce state variance? Where does it flatline?
8. How do we compare global_bot, particle map and true maps?
9. Why does the particle map differentiate significantly from n=3 to n=4 msmts at the same spot (why not n=2)?
10. Need an extra control parameter to control the Gaussian blurring function. 
At the moment, the correlation length and the variance for the blur are the same. 
But the blurring kernel (and its hyperparams) should be tuned independently. 
11. Gaussian blurring needs to change (either by incorporating global_bot / reducing avergaing as more real data is avail)



'''

As measurements increase, the scan ranges can be reduced such that quasi-measurements have a lesser impact on the inference procedure compared to actual measurements. This diminsishing scan-range functionality has not yet been implemented

In fact, the treatment of re-observation in standard SLAM (such that all features become correlated over time) is displaced by the need for quasi-measurements in our application.  This means that the simultaneous mapping and localisation benefits that accrue from re-observing map features - are not being utilised.This is a key question that needs to be explored with more care.


Extensions (not implemented):

 - Every particle in the particle filter should have a randomised scan range, and the weighting procedure should pick out the most relevant scan range based on the likelihood function
 - Define what it means to "re-observe" a map feature in SLAM for this application - is there any benefit to doing SLAM?
 - Change the likelihood function based on second paper
 - Jitter in dephasing noise map ==> model this as imperfect knoweldge of robot position on the grid.
 - Time domain dynamics of map ==> add a temporal AR process to evolve the map at each time step
 

In [ ]:
with suppress_stdout():
    slamize.qslam_run(dunk, loopcontrols)

In [ ]:
'''From a Single Posterior State Particle'''
print slamize.particles[0].r_questbk

In [ ]:
'''From Global Bot (Scanner)'''
print slamize.global_bot.r_guestbk_counter
print slamize.global_bot.r_questbk

In [ ]:
'''Compare True Maps to Maps from Particle (Quasi and Physical Msmts) and Global Bot (Physical)'''
print "True Map"
print map_
print
print "Particle Map"
print slamize.particles[0].m_vals
print 
print "Global Bot Map"
inferred_m_vals = np.asarray([slamize.particles[0].get_phase_method(val) for val in slamize.global_bot.r_questbk.flatten()]).reshape(5,5)
print inferred_m_vals


'''
Notes
1. Where the bot has NOT physically visited (off control path), global_bot.m_vals == pi and partcicles[i].m_vals == 0 are equivalent
2. For zero correlation length in the control path, both maps should be the same
3. For physically measured qubits, both maps should be the same asympotically as long thigns are "continuous"

'''

In [ ]:
x = np.linspace(0,1,100)
y = np.arccos(2.0*x - 1.)
plt.plot()
plt.plot(x,y)
plt.xlabel('Prob')
plt.ylabel('Phase (rad)')
plt.show()

In [ ]:
R1=1.42
R2=5
x = np.linspace(0,5,1000)
y1 = np.exp(-(x)**2 / 2.0*(1./(R1 + 1e-14))**2)
y2 = np.exp(-(x)**2 / 2.0*(1./(R2 + 1e-14))**2)
plt.plot()
plt.plot(x,y1, 'k',label='Short R='+str(R1))
plt.plot(x, y2, 'b', label='Long R='+str(R2))
plt.xlabel('Separation')
plt.ylabel('Correlation')
plt.legend(loc=0)
plt.show()

In [3]:
import numpy as np
missing_data = np.nan?

In [5]:
missing_data = np.nan_to_num?
print missing_data

nan


In [13]:
missing_data = np.nan*np.zeros((5,5))
missing_data[2,2] = 2.

In [15]:
np.nan_to_num(missing_data**2)

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  4.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])